In [1]:
# Initial imports
import os
import json
import requests
import pandas as pd
from pathlib import Path
import quandl
from dotenv import load_dotenv

%matplotlib inline

In [2]:
load_dotenv()

True

In [3]:
# Set API key from env variables
quandl.ApiConfig.api_key = os.getenv("QUANDL_API")

## Indicators
The indicators table explains how the housing data has been categorized

In [4]:
# Download housing data from quadl api
indicator_df = quandl.get_table("ZILLOW/INDICATORS")

In [5]:
# Show indicators dataframe
indicator_df.head(20)

,indicator_id,indicator,category
None,,,
0,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
1,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
2,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
3,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values
5,Z5BR,ZHVI 5+ Bedroom Time Series ($),Home values
6,Z4BR,ZHVI 4-Bedroom Time Series ($),Home values
7,Z3BR,ZHVI 3-Bedroom Time Series ($),Home values
8,Z2BR,ZHVI 2-Bedroom Time Series ($),Home values


## Regions
The regions table explains the different region types for the housing data

In [6]:
# Show regions dataframe
regions = quandl.get_table("ZILLOW/REGIONS", paginate=True)
regions.head()

,region_id,region_type,region
None,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin
1,99998,zip,98846; WA; Okanogan County; Pateros
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades
3,99996,zip,98844; WA; Okanogan County; Oroville
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo


## Data Table
The data table contains all the housing data by indicator and region.

The downloaded zip is extracted to a location outside of the repository due to it's large size.

In [9]:
# -- Code commented out as it only needs to be run a single time

# -------
# data = quandl.export_table('ZILLOW/DATA', filename="../../Repositories/local/zdata.zip")
# with zipfile.ZipFile('../../Repositories/local/zdata.zip', 'r') as zip_ref:
#    zip_ref.extractall('../../Repositories/local/')
# os.rename('../../Repositories/local/ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv', '../../Repositories/local/zdata.csv')
# Read extracted zip file into a dataframe
zdata = pd.read_csv('../../Repositories/local/zdata.csv')
# -------

In [10]:
zdata.shape

(127237469, 4)

In [11]:
regions.region_type.value_counts()

zip       31172
city      27471
neigh     16711
county     2887
metro       915
state        51
Name: region_type, dtype: int64

## Extract metro codes to list

In [12]:
# Casting the list values as int for use to make splits from the region_id column
metros = regions[regions.region_type == 'metro']
metros.tail()

,region_id,region_type,region
None,,,
51628,394300,metro,"Ada, OK; OK"
51630,394299,metro,"Abilene, TX; TX"
51631,394298,metro,"Aberdeen, WA; WA"
51632,394297,metro,"Aberdeen, SD; SD"
78827,102001,metro,United States


In [13]:
metro_codes = [int(i) for i in metros.region_id.tolist()]
metro_codes[:5]

[753929, 753928, 753927, 753926, 753925]

## Extract home value codes to list

In [14]:
home_value_codes = []
for index, row in indicator_df.iterrows():
    if row[2] == "Home values":
        home_value_codes.append(row[0])
home_value_codes

['ZSFH',
 'ZCON',
 'ZATT',
 'ZALL',
 'ZABT',
 'Z5BR',
 'Z4BR',
 'Z3BR',
 'Z2BR',
 'Z1BR']

In [15]:
del home_value_codes[0]
del home_value_codes[1:4]
home_value_codes

['ZCON', 'Z5BR', 'Z4BR', 'Z3BR', 'Z2BR', 'Z1BR']

## Home values used to create a slice of zdata

In [16]:
# Dataframe containing all homes from 1BR to 5BR and Condos
homes = zdata[zdata.indicator_id.isin(home_value_codes)]
homes

,indicator_id,region_id,date,value
11667,ZCON,9,2020-10-31,537729.0
11960,ZCON,54,2020-10-31,185435.0
12253,ZCON,43,2020-10-31,580421.0
12546,ZCON,14,2020-10-31,196766.0
12839,ZCON,21,2020-10-31,202002.0
...,...,...,...,...
127237464,Z4BR,49589,2020-02-29,103928.0
127237465,Z4BR,49589,2020-03-31,105018.0
127237466,Z4BR,49589,2020-04-30,105616.0
127237467,Z4BR,49589,2020-06-30,105040.0


## Metro codes used to slice homes data

In [74]:
metro_homes = homes[homes.region_id.isin(metro_codes)]
metro_homes

,indicator_id,region_id,date,value
1553170,ZCON,102001,2020-06-30,272122.0
1553174,ZCON,394913,2020-06-30,498772.0
1553179,ZCON,753899,2020-06-30,547500.0
1553182,ZCON,394463,2020-06-30,206792.0
1553187,ZCON,394514,2020-06-30,184800.0
...,...,...,...,...
51837197,Z4BR,394743,2019-10-31,379724.0
51837198,Z4BR,394743,2019-11-30,380707.0
51837199,Z4BR,394743,2019-12-31,381119.0
51837200,Z4BR,394743,2020-01-31,381255.0


In [75]:
# Reset the index for the new dataframe before writing to csv
metro_homes = metro_homes.reset_index(drop=True)
metro_homes.sort_values(by='date', inplace=True)
metro_homes

,indicator_id,region_id,date,value
534143,Z2BR,394314,1996-01-31,28575.0
764998,Z3BR,394949,1996-01-31,73795.0
480862,Z2BR,394308,1996-01-31,86397.0
562650,Z2BR,394501,1996-01-31,46682.0
843860,Z3BR,395178,1996-01-31,84852.0
...,...,...,...,...
59337,ZCON,394447,2021-05-31,117956.0
70443,Z1BR,394734,2021-05-31,82249.0
59334,ZCON,394471,2021-05-31,223577.0
59349,ZCON,394645,2021-05-31,414356.0


In [76]:
metro_homes.to_csv('./Resources/metro_homes.csv', index=False,)
metro_homes

,indicator_id,region_id,date,value
534143,Z2BR,394314,1996-01-31,28575.0
764998,Z3BR,394949,1996-01-31,73795.0
480862,Z2BR,394308,1996-01-31,86397.0
562650,Z2BR,394501,1996-01-31,46682.0
843860,Z3BR,395178,1996-01-31,84852.0
...,...,...,...,...
59337,ZCON,394447,2021-05-31,117956.0
70443,Z1BR,394734,2021-05-31,82249.0
59334,ZCON,394471,2021-05-31,223577.0
59349,ZCON,394645,2021-05-31,414356.0


In [77]:
metro_condo = metro_homes[metro_homes.indicator_id == 'ZCON']
metro_condo.sort_values(by='date', inplace=True)
metro_condo

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,indicator_id,region_id,date,value
96350,ZCON,394619,1996-01-31,70863.0
149890,ZCON,394314,1996-01-31,85074.0
106031,ZCON,394528,1996-01-31,105987.0
195349,ZCON,394764,1996-01-31,49046.0
81023,ZCON,394463,1996-01-31,135785.0
...,...,...,...,...
59964,ZCON,394580,2021-05-31,147685.0
59925,ZCON,394709,2021-05-31,190761.0
59922,ZCON,394599,2021-05-31,180417.0
60021,ZCON,394907,2021-05-31,196293.0


In [78]:
metro_1br = metro_homes[metro_homes.indicator_id == 'Z1BR']
metro_1br.sort_values(by='date', inplace=True)
metro_1br

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,indicator_id,region_id,date,value
1042909,Z1BR,394891,1996-01-31,42905.0
1061713,Z1BR,753892,1996-01-31,26374.0
978424,Z1BR,394788,1996-01-31,87144.0
1007727,Z1BR,394829,1996-01-31,60190.0
889453,Z1BR,394463,1996-01-31,99397.0
...,...,...,...,...
71740,Z1BR,394886,2021-05-31,105399.0
71743,Z1BR,394962,2021-05-31,57716.0
71746,Z1BR,394491,2021-05-31,96095.0
70065,Z1BR,394346,2021-05-31,171744.0


In [79]:
metro_2br = metro_homes[metro_homes.indicator_id == 'Z2BR']
metro_2br.sort_values(by='date', inplace=True)
metro_2br

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,indicator_id,region_id,date,value
534143,Z2BR,394314,1996-01-31,28575.0
508618,Z2BR,394808,1996-01-31,40731.0
670749,Z2BR,395236,1996-01-31,20735.0
519080,Z2BR,395002,1996-01-31,103644.0
605836,Z2BR,394632,1996-01-31,38137.0
...,...,...,...,...
65304,Z2BR,395205,2021-05-31,154572.0
64438,Z2BR,394488,2021-05-31,58797.0
64417,Z2BR,394579,2021-05-31,100221.0
64474,Z2BR,394447,2021-05-31,122964.0


In [80]:
metro_3br = metro_homes[metro_homes.indicator_id == 'Z3BR']
metro_3br.sort_values(by='date', inplace=True)
metro_3br

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,indicator_id,region_id,date,value
764998,Z3BR,394949,1996-01-31,73795.0
687949,Z3BR,394669,1996-01-31,144998.0
713829,Z3BR,394875,1996-01-31,93283.0
818861,Z3BR,394556,1996-01-31,230000.0
683640,Z3BR,394475,1996-01-31,103488.0
...,...,...,...,...
66803,Z3BR,394849,2021-05-31,162608.0
68789,Z3BR,394801,2021-05-31,105781.0
69140,Z3BR,394364,2021-05-31,66896.0
66800,Z3BR,394714,2021-05-31,245144.0


In [81]:
metro_4br = metro_homes[metro_homes.indicator_id == 'Z4BR']
metro_4br.sort_values(by='date', inplace=True)
metro_4br

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,indicator_id,region_id,date,value
1176965,Z4BR,753890,1996-01-31,120535.0
1131475,Z4BR,394572,1996-01-31,104786.0
1245249,Z4BR,753865,1996-01-31,108239.0
1224486,Z4BR,394933,1996-01-31,44112.0
1174883,Z4BR,394949,1996-01-31,116969.0
...,...,...,...,...
74056,Z4BR,394500,2021-05-31,259472.0
74053,Z4BR,753865,2021-05-31,253778.0
74050,Z4BR,394409,2021-05-31,87370.0
74131,Z4BR,753916,2021-05-31,541437.0


In [82]:
metro_5br = metro_homes[metro_homes.indicator_id == 'Z5BR']
metro_5br.sort_values(by='date', inplace=True)
metro_5br

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,indicator_id,region_id,date,value
281576,Z5BR,394997,1996-01-31,188033.0
262000,Z5BR,395059,1996-01-31,453298.0
358242,Z5BR,395086,1996-01-31,178269.0
255951,Z5BR,395025,1996-01-31,184235.0
310182,Z5BR,394848,1996-01-31,175141.0
...,...,...,...,...
62523,Z5BR,394988,2021-05-31,344888.0
62520,Z5BR,394941,2021-05-31,267785.0
62517,Z5BR,394859,2021-05-31,316594.0
63891,Z5BR,395236,2021-05-31,214734.0


In [83]:
metro_condo.reset_index(drop=True)

,indicator_id,region_id,date,value
0,ZCON,394619,1996-01-31,70863.0
1,ZCON,394314,1996-01-31,85074.0
2,ZCON,394528,1996-01-31,105987.0
3,ZCON,394764,1996-01-31,49046.0
4,ZCON,394463,1996-01-31,135785.0
...,...,...,...,...
184027,ZCON,394580,2021-05-31,147685.0
184028,ZCON,394709,2021-05-31,190761.0
184029,ZCON,394599,2021-05-31,180417.0
184030,ZCON,394907,2021-05-31,196293.0


In [84]:
metro_condo.to_csv('./Resources/metro_condo.csv', index=False)

In [85]:
metro_1br.reset_index(drop=True)

,indicator_id,region_id,date,value
0,Z1BR,394891,1996-01-31,42905.0
1,Z1BR,753892,1996-01-31,26374.0
2,Z1BR,394788,1996-01-31,87144.0
3,Z1BR,394829,1996-01-31,60190.0
4,Z1BR,394463,1996-01-31,99397.0
...,...,...,...,...
210181,Z1BR,394886,2021-05-31,105399.0
210182,Z1BR,394962,2021-05-31,57716.0
210183,Z1BR,394491,2021-05-31,96095.0
210184,Z1BR,394346,2021-05-31,171744.0


In [86]:
metro_1br.to_csv('./Resources/metro_1br.csv', index=False)

In [87]:
metro_2br.reset_index(drop=True)

,indicator_id,region_id,date,value
0,Z2BR,394314,1996-01-31,28575.0
1,Z2BR,394808,1996-01-31,40731.0
2,Z2BR,395236,1996-01-31,20735.0
3,Z2BR,395002,1996-01-31,103644.0
4,Z2BR,394632,1996-01-31,38137.0
...,...,...,...,...
225351,Z2BR,395205,2021-05-31,154572.0
225352,Z2BR,394488,2021-05-31,58797.0
225353,Z2BR,394579,2021-05-31,100221.0
225354,Z2BR,394447,2021-05-31,122964.0


In [88]:
metro_condo.to_csv('./Resources/metro_2br.csv', index=False)

In [89]:
metro_3br.reset_index(drop=True)

,indicator_id,region_id,date,value
0,Z3BR,394949,1996-01-31,73795.0
1,Z3BR,394669,1996-01-31,144998.0
2,Z3BR,394875,1996-01-31,93283.0
3,Z3BR,394556,1996-01-31,230000.0
4,Z3BR,394475,1996-01-31,103488.0
...,...,...,...,...
227046,Z3BR,394849,2021-05-31,162608.0
227047,Z3BR,394801,2021-05-31,105781.0
227048,Z3BR,394364,2021-05-31,66896.0
227049,Z3BR,394714,2021-05-31,245144.0


In [90]:
metro_3br.to_csv('./Resources/metro_3br.csv', index=False)

In [91]:
metro_4br.reset_index(drop=True)

,indicator_id,region_id,date,value
0,Z4BR,753890,1996-01-31,120535.0
1,Z4BR,394572,1996-01-31,104786.0
2,Z4BR,753865,1996-01-31,108239.0
3,Z4BR,394933,1996-01-31,44112.0
4,Z4BR,394949,1996-01-31,116969.0
...,...,...,...,...
225980,Z4BR,394500,2021-05-31,259472.0
225981,Z4BR,753865,2021-05-31,253778.0
225982,Z4BR,394409,2021-05-31,87370.0
225983,Z4BR,753916,2021-05-31,541437.0


In [92]:
metro_4br.to_csv('./Resources/metro_4br.csv', index=False)

In [93]:
metro_5br.reset_index(drop=True)

,indicator_id,region_id,date,value
0,Z5BR,394997,1996-01-31,188033.0
1,Z5BR,395059,1996-01-31,453298.0
2,Z5BR,395086,1996-01-31,178269.0
3,Z5BR,395025,1996-01-31,184235.0
4,Z5BR,394848,1996-01-31,175141.0
...,...,...,...,...
225020,Z5BR,394988,2021-05-31,344888.0
225021,Z5BR,394941,2021-05-31,267785.0
225022,Z5BR,394859,2021-05-31,316594.0
225023,Z5BR,395236,2021-05-31,214734.0


In [94]:
metro_5br.to_csv('./Resources/metro_5br.csv', index=False)